In [20]:
#获取索引

import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

gheaders = {
    'Referer': 'https://ieeexplore.ieee.org/xpl/issues?punumber=35&isnumber=9161976',#网站反爬
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
    }
res = requests.get('https://ieeexplore.ieee.org/rest/publication/35/regular-issues',headers=gheaders)
data = res.text

volume = re.findall('"volume":"(.*?)","issue":".*?","issueNumber":.*?,.*?"year":".*?","month":".*?"',data)
issue = re.findall('"volume":".*?","issue":"(.*?)","issueNumber":.*?,.*?"year":".*?","month":".*?"',data)
issuenumber = re.findall('"volume":".*?","issue":".*?","issueNumber":(.*?),.*?.*?"year":".*?","month":".*?"',data)
year = re.findall('"volume":".*?","issue":".*?","issueNumber":.*?,.*?"year":"(.*?)","month":".*?"',data)
month = re.findall('"volume":".*?","issue":".*?","issueNumber":.*?,.*?"year":".*?","month":"(.*?)"',data)

data_all = {
    'volume':np.array(volume),
    'year':np.array(year),
    'month':np.array(month),
    'issue':np.array(issue),
    'issuenumber':np.array(issuenumber)
}
da = pd.DataFrame(data_all)
da

,volume,year,month,issue,issuenumber
0,58,2020,July,7,9161976
1,58,2020,June,6,9141177
2,58,2020,May,5,9112723
3,58,2020,April,4,9071969
4,58,2020,March,3,9040205
...,...,...,...,...,...
470,17,1979,September,5,23727
471,17,1979,July,4,23726
472,17,1979,May,3,23725
473,17,1979,March,2,23724


In [126]:
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


def find_year(da):
    return da.index.str.startswith("2020")==1

#存储为excel
writer = pd.ExcelWriter(r'D:/jupyter_save/data.xlsx')
da['date'] = da['year'].map(str) + "-" + da['month']
da.set_index('date', inplace = True)
#str.replace('*','*')
da.loc[:,'issuenumber'] = da['issuenumber'].astype('int64')
da.to_excel(writer,sheet_name = 'IEEE Communications Magazine',index=True)
writer.save()
#数据爬取处理预备
file = pd.ExcelFile(r'./data.xlsx')
handle = pd.read_excel('./data.xlsx')
issuenum = handle['issuenumber']
datestr = handle['date'] 
#test
da.loc[find_year,:]


,volume,year,month,issue,issuenumber
date,,,,,
2020-July,58,2020,July,7,9161976
2020-June,58,2020,June,6,9141177
2020-May,58,2020,May,5,9112723
2020-April,58,2020,April,4,9071969
2020-March,58,2020,March,3,9040205
2020-February,58,2020,February,2,8999408
2020-January,58,2020,January,1,8970148


In [7]:
# import requests
# import re
# import pandas as pd
# import numpy as np
# import xlwt
# from lxml import etree
# import json
# #测试命令专用
# # issuenum = handle.loc[:,'issuenumber']
# # type(issuenum)
# # lenth = len(issuenum)
# # lenth
# # t=issuenum.loc[0]
# # t
# # y = '12434\n12'
# # t=pd.Series([0])
# # t.loc[0]=1
# # print(y)
# t1 = pd.Series(['1','2'])
# t2 = pd.Series(['2','5'])
# pd.concat([t1,t2],axis=0,join = 'outer',ignore_index = False)

0    1
1    2
0    2
1    5
dtype: object

In [ ]:
from lxml import etree
import json
import time
#id储存在toc数据包
length=len(issuenum)

alf = open(r'save401-500.txt', 'w')

def get_paperinfo(issuenumber):
    jsonerror_flag = 0
    name = pd.Series([0])
    for x in range(400,475):#一次传太多容易报错，改这里，有多少个运行上面第一栏代码可以得到，筛选最新的
        time.sleep(1)
        print(x)
        y = issuenumber.loc[x]
        y = str(y)
        gheaders = {
            'Host': 'ieeexplore.ieee.org',
            'Referer': 'https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber='+y+'&punumber=35',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
            }
        toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/35/issue/'+y+'/toc'
        for page in range(0,3):
            if page==1:
                payload = '{"isnumber":"'+y+'","punumber":"35","sortType":"vol-only-seq","pageNumber":"2"}'
            elif page==2:
                payload = '{"isnumber":"'+y+'","punumber":"35","sortType":"vol-only-seq","pageNumber":"3"}'
            else:
                payload = '{"isnumber":"'+y+'","punumber":"35","sortType":"vol-only-seq"}'     
            toc_res = requests.post(toc_url, headers = gheaders,data = payload)
            try:
                toc_dic = json.loads(toc_res.text)
                try:
                    articles = toc_dic['records']
                except KeyError:
                    print('不存在第'+str(page+1)+'页')
                    break
                else:
                    for article in articles:
                        flag = article['ephemera']
                        if flag == False:
                            #title = article['highlightedTitle']
                            paperid = article['articleNumber']
                            alf.write(paperid+'\n')
                            print(paperid)
            except json.JSONDecodeError:
                print('JSON error,try again-------issuenumber=%s', y )
                toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/35/issue/'+y+'/toc'
                payload = '{"isnumber":"'+y+'","punumber":"35","sortType":"vol-only-seq"}'
                toc_res = requests.post(toc_url, headers = gheaders,data = payload)
                try:
                    toc_dic = json.loads(toc_res.text)
                    try:
                        articles = toc_dic['records']
                    except KeyError:
                        print('KEY_ERROR，不存在第'+str(page+1)+'页')
                        break
                    else:
                        for article in articles:
                            flag = article['ephemera']
                            if flag == False:
                                #title = article['highlightedTitle']
                                paperid = article['articleNumber']
                                alf.write(paperid+'\n')
                                print(paperid)
                except json.JSONDecodeError:
                    name.loc[jsonerror_flag] = y
                    jsonerror_flag = jsonerror_flag + 1
                    print('Error  again,Pass\n')
                    continue
                else:
                    continue
            else:
                continue
    alf.close()
get_paperinfo(issuenum)
        


    



In [ ]:
#报错后补充数据


#         from lxml import etree
#         import json
#         addtxt = open(r'add.txt', 'w')
#         gheaders = {
#             'Host': 'ieeexplore.ieee.org',
#             'Referer': 'https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=9161976&punumber=35',
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
#             }
#         toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/35/issue/'+'9161976'+'/toc'
#         payload = '{"isnumber":"'+'9161976'+'","punumber":"35","sortType":"vol-only-seq"}'
#         toc_res = requests.post(toc_url, headers = gheaders,data = payload)
#         toc_dic = json.loads(toc_res.text)
#         toc_dic['records']
#         articles = toc_dic['records']
#         for article in articles:
#             flag = article['ephemera']
#             if flag == False:
#                 title = article['highlightedTitle']
#                 paperid = article['articleNumber']
#                 alf.write(title.replace('\n','')+'>_<'+paperid+'\n')
#                 print(title)
#                 print(paperid)
#                 import time
#                 time.sleep(1)
#         addtxt.close()

In [137]:
#对得到的id整合
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
#pd.concat(objs,axis=0,join = 'outer',ignore_index = False)outer索引对齐方式
#id_data_part1 = pd.read_csv(r'./save1-100.txt',index_col=0)
#DataFrame.append(other,ignore_index=False)Ture忽略掉原来的索引
id_data_part1 = pd.read_csv(r'./save1-100.txt',header=None)
id_data_part2 = pd.read_csv(r'./save101-200.txt',header=None)
id_data_part3 = pd.read_csv(r'./save201-300.txt',header=None)
id_data_part4 = pd.read_csv(r'./save301-400.txt',header=None)
id_data_part5 = pd.read_csv(r'./save401-500.txt',header=None)
id_data=pd.concat([id_data_part1,id_data_part2,id_data_part3,id_data_part4,id_data_part5],axis=0,join = 'inner',ignore_index = True )
#id_data.loc[96:104]
id_data.rename(columns={0:'paper-ID'}, inplace = True)
writer_paperid = pd.ExcelWriter(r'D:/jupyter_save/paperid.xlsx')
id_data.to_excel(writer_paperid,sheet_name = 'paper_idsave',index=True)
writer_paperid.save()
print('Finish')

Finish


In [59]:
# #test
# import requests
# import re
# import pandas as pd
# import numpy as np
# import xlwt
# from lxml import etree
# import json

# id_test = 9161991
# gheaders = {
# 'Host': 'ieeexplore.ieee.org',
# 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
# 'Referer': 'https://ieeexplore.ieee.org/document/9161991/keywords',
# }
# metadata_url='https://ieeexplore.ieee.org/document/9161991/keywords'
# meta_res = requests.get(metadata_url,headers=gheaders)
# info_data = meta_res.text
# #global.document.metadata.*?"Keywords","kwd":(.*?)}],"isGetArticle".*?"journalDisplayDateOfPublication":.*?,"isPromo"
# keywords = re.findall('"IEEE Keywords","kwd":(.*?)}]',info_data)
# pub=re.findall('"journalDisplayDateOfPublication":(.*?),',info_data)
# paper=re.findall('<title>(.*?) - IEEE',info_data)
# keyword=''.join(keywords)
# paper_name=''.join(paper)
# pub=''.join(pub)
# pub_date=pub.replace('"','')
# keyword=keyword.replace('"','')
# keyword=keyword.replace('[','')
# keyword=keyword.replace(']','')
# keyword_all=keyword.split(',')
# pub_date=pub_date.split(' ')




'A Hierarchical and Modular Radio Resource Management Architecture for 5G and Beyond'

In [70]:
# #test
# import requests
# import re
# import pandas as pd
# import numpy as np
# import xlwt
# from lxml import etree
# import json

# info_creat = pd.DataFrame({'Year':[],
#                           'Month':[],
#                           'Day':[],
#                           'Name':[],
#                           'Keyword_0':[],
#                           'Keyword_1':[],
#                           'Keyword_2':[],
#                           'Keyword_3':[],
#                           'Keyword_4':[]                                                       
#                          })
# table_creat = pd.DataFrame({'Year':[pub_date[2]],
#                           'Month':[pub_date[1]],
#                           'Day':[pub_date[0]],
#                           'Name':[paper_name],                                                   
#                          })
# for i in range(0,len(keyword_all)):
#     name_i = str(i)
#     table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]


# info_save = pd.concat([info_creat,table_creat],ignore_index = True)
# info_save


,Year,Month,Day,Name,Keyword_0,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5,Keyword_6
0,2020,August,07,A Hierarchical and Modular Radio Resource Mana...,5G mobile communication,Resource management,Complexity theory,Wireless communication,Heterogeneous networks,Licenses,Long Term Evolution


In [138]:
#对所有文章进行信息获取
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

id_handle = pd.read_excel(r'./paperid.xlsx')
id_data = id_handle['paper-ID']
len(id_data)

6513

In [25]:
#对所有文章进行信息获取
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

id_handle = pd.read_excel(r'./paperid.xlsx')
id_data = id_handle['paper-ID']
#id_data
info_creat = pd.DataFrame({'Year':[],
                          'Month':[],
                          'Day':[],
                          'Name':[],
                          'Keyword_0':[],
                          'Keyword_1':[],
                          'Keyword_2':[],
                          'Keyword_3':[],
                          'Keyword_4':[]                                                       
                         })
info_save = info_creat
def get_info(id_data, info_save,info_creat):
    median_info = info_creat
    print('一组20篇文章，组数如下')
    for x in range(325,326):
        times = x+1
        print(times)
        for y in range(0,13):
            flag = x*20 + y
            id_test = id_data.loc[flag]
            id_test = str(id_test)
            gheaders = {
            'Host': 'ieeexplore.ieee.org',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Referer': 'https://ieeexplore.ieee.org/document/'+id_test+'/keywords',
            }
            metadata_url='https://ieeexplore.ieee.org/document/'+id_test+'/keywords'
            #try看情况是否要添加
            meta_res = requests.get(metadata_url,headers=gheaders)
            info_data = meta_res.text
            #global.document.metadata.*?"Keywords","kwd":(.*?)}],"isGetArticle".*?"journalDisplayDateOfPublication":.*?,"isPromo"
            keywords = re.findall('"IEEE Keywords","kwd":(.*?)]}',info_data)
            pub=re.findall('"journalDisplayDateOfPublication":(.*?),',info_data)
            paper=re.findall('<title>(.*?) - IEEE',info_data)
            keyword=''.join(keywords)
            paper_name=''.join(paper)
            pub=''.join(pub)
            pub_date=pub.replace('"','')
            keyword=keyword.replace('"','')
            keyword=keyword.replace('[','')
            keyword=keyword.replace(']','')
            keyword_all=keyword.split(',')
            pub_date=pub_date.split(' ')
            table_creat = pd.DataFrame({'Year':[pub_date[2]],
                                        'Month':[pub_date[1]],
                                        'Day':[pub_date[0]],
                                        'Name':[paper_name],                                                   
                                       })
            for i in range(0,len(keyword_all)):
                name_i = str(i)
                table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]
                #备忘：这里是当前文章的一行信息
            median_info = pd.concat([median_info,table_creat],ignore_index = True)
            #备忘：median_info循环20遍记录数据
        info_save = pd.concat([info_save,median_info],ignore_index = True)
        median_info = info_creat
        print('第'+str(times)+'组完成')
    return info_save 
info_data = get_info(id_data, info_save,info_creat)
paper_info = pd.ExcelWriter(r'D:/jupyter_save/paper_info.xlsx')
info_data.to_excel(paper_info,sheet_name = 'IEEE Communications Magazine',index=False)
paper_info.save()
runned_flag = True
print('Finish')

一组20篇文章，组数如下
326
第326组完成
Finish


In [15]:
runned_flag

False

In [ ]:
info_data

In [26]:
#分批录入

import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
if runned_flag ==True:
    merged_writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
    merge_info_now = pd.read_excel(r'./paper_info.xlsx')
    merge_info_pro = pd.read_excel(r'./total_info.xlsx')
    merge_allinfo = pd.concat([merge_info_pro,merge_info_now],ignore_index = True)
    merge_allinfo.to_excel(merged_writer,sheet_name = 'IEEE Communications Magazine',index=False)
    merged_writer.save()
    runned_flag =False
    print('Finish')
else:
    print('无需重复录入')

Finish


In [139]:
# #建立此期刊对应sheet且会初始化！！！
# import requests
# import re
# import pandas as pd
# import numpy as np
# import xlwt
# from lxml import etree
# import json

# This = pd.DataFrame({'Year':[],
#                           'Month':[],
#                           'Day':[],
#                           'Name':[],
#                           'Keyword_0':[],
#                           'Keyword_1':[],
#                           'Keyword_2':[],
#                           'Keyword_3':[],
#                           'Keyword_4':[]                                                       
#                          })
# writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
# This.to_excel(writer,sheet_name = 'IEEE Communications Magazine',index=False)
# writer.save()
# print('Finish')


Finish


In [123]:
#     #测试用

#     import requests
#     import re
#     import pandas as pd
#     import numpy as np
#     import xlwt
#     from lxml import etree
#     import json
#     merged_writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
#     merge_info_now = pd.read_excel(r'./paper_info.xlsx')
#     merge_info_pro = pd.read_excel(r'./total_info.xlsx')
#     merge_allinfo = pd.concat([merge_info_pro,merge_info_now],ignore_index = True)
#     merge_allinfo.to_excel(merged_writer,sheet_name = 'IEEE Communications Magazine',index=False)
#     merged_writer.save()
    
#     print('Finish')
#     merge_info_pro

Finish


,Year,Month,Day,Name,Keyword_0,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5,Keyword_6,Keyword_7
0,2020,August,7,"Wake-up Radio-Based 5G Mobile Access: Methods,...",Power demand,Synchronization,Delays,5G mobile communication,Energy consumption,Batteries,3GPP,NaN
1,2020,August,7,Advanced Dynamic Spectrum 5G Mobile Networks E...,Sensors,Resource management,Europe,5G mobile communication,Synthetic aperture sonar,Broadband communication,3GPP,NaN
2,2020,August,7,A Hierarchical and Modular Radio Resource Mana...,5G mobile communication,Resource management,Complexity theory,Wireless communication,Heterogeneous networks,Licenses,Long Term Evolution,NaN
3,2020,August,7,BAGKD: A Batch Authentication and Group Key Di...,Authentication,Protocols,Vehicular ad hoc networks,Public key,Road transportation,NaN,NaN,NaN
4,2020,August,7,Automating Vertical Services Deployments over ...,5G mobile communication,Industries,Network slicing,Cloud computing,Systems architecture,Software defined networking,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
75,2020,June,9,A Hybrid Positioning System for Location-Based...,Bluetooth,Protocols,Servers,Wireless fidelity,Zigbee,Robustness,Position measurement,NaN
76,2020,April,20,Toward Operator-to-Waveform 5G Radio Access Ne...,5G mobile communication,IP networks,Radio access networks,Complexity theory,Quality of experience,Network slicing,NaN,NaN
77,2020,April,20,LightEdge: Mapping the Evolution of Multi-Acce...,5G mobile communication,IP networks,Edge computing,Internet,Computer architecture,3GPP,NaN,NaN
78,2020,April,20,End-to-End Network Slicing for Flash Crowds,Network slicing,Wireless communication,Virtualization,5G mobile communication,Software engineering,Cloud computing,NaN,NaN


In [112]:
merge_info_pro

,Unnamed: 0,Year,Month,Day,Name,Keyword_0,Keyword_1,Keyword_2,Keyword_3,Keyword_4


In [17]:
#统一不同时期的格式并存储
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


writer = pd.ExcelWriter(r'D:/jupyter_save/Final_info.xlsx')
info = pd.read_excel(r'./total_info.xlsx')
info['Date'] = info['Month'].map(str)  + info['Day'].map(str)
info_fix = info['Date'].str.replace("[0123456789.]","").str.replace('nan','')
info_fix = info_fix.str.replace('Jan','01').str.replace('Feb','02').str.replace('Mar','03').str.replace('Apr','04').str.replace('May','05').str.replace('Nov','11')
info_fix = info_fix.str.replace('Jun','06').str.replace('Jul','07').str.replace('Aug','08').str.replace('Sep','09').str.replace('Oct','10').str.replace('Dec','12')
info_fix = info_fix.str.replace("[abcehilmorstuy]","")
info['Date'] = info_fix
info['Date'] = info['Year'].map(str)  +'-'+ info['Date'].map(str)
# info['Date'] = info['Date'].str.replace('\n','')
del info['Month']
del info['Day']
del info['Year']
info.set_index('Date',inplace = True)
info.to_excel(writer,sheet_name = 'IEEE Communications Magazine',index=True)
writer.save()
# da.loc[:,'issuenumber'] = da['issuenumber'].astype('int64')
# da.to_excel(writer,sheet_name = 'IEEE Communications Magazine',index=True)
# writer.save()



# merged_writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
# merge_info_now = pd.read_excel(r'./paper_info.xlsx')
# merge_info_pro = pd.read_excel(r'./total_info.xlsx')
# merge_allinfo = pd.concat([merge_info_pro,merge_info_now],ignore_index = True)
# merge_allinfo.to_excel(merged_writer,sheet_name = 'IEEE Communications Magazine',index=False)
# merged_writer.save()